<h3>VVI</h3>

In [2]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from langchain.agents import AgentType, initialize_agent
from langchain.chains import LLMMathChain
from pydantic import BaseModel,Field
from langchain_core.tools import Tool, StructuredTool
from langchain import hub
from langchain.agents import (create_openai_tools_agent,create_tool_calling_agent,
                        AgentExecutor,create_openai_functions_agent)

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [7]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

In [8]:
llmMathChain=LLMMathChain.from_llm(llm=llmOpenAI,verbose=True)

In [9]:
llmMathChain.invoke(input={"What is 34 to the power 2 plus 43^2?"})



> Entering new LLMMathChain chain...
{'What is 34 to the power 2 plus 43^2?'}```text
34**2 + 43**2
```
...numexpr.evaluate("34**2 + 43**2")...

Answer: 3005
> Finished chain.


{'question': {'What is 34 to the power 2 plus 43^2?'},
 'answer': 'Answer: 3005'}

In [10]:
# Define Tools which provides 1) The nth prime number and 2) LLMMathChain to act as the calculator

primes={998:7901, 999: 7907, 1000: 7919}

##########----------Class Which utilizes in the MathsChain----------############
class CalculatorInput(BaseModel):
    question:str=Field(description="Question to ask to the Calculator")

##########-----------Class for Finding the llm_math_chain ---------------############

def getPrime(n: int, primes:dict=primes) ->str:
    return str(primes.get(int(n)))

async def agetPrime(n: int, primes:dict=primes) ->str:
    return str(primes.get(int(n)))

class PrimeInput(BaseModel):
    n:int=Field()

In [11]:
tools=[
    Tool(
        name="GetPrime",
        func=getPrime,
        description="A Tool that returns the `n`th prime number",
        args_schema=PrimeInput,
        coroutine=agetPrime
    ),
    Tool.from_function(
        name="Calculator",
        description="Useful when you would need to compute mathematical expressions",
        func=llmMathChain.invoke,
        args_schema=CalculatorInput,
        coroutine=llmMathChain.invoke
    )
]


In [12]:
def isPrime(n:int) -> bool:
    if n<=1 or (n%2==0 and n>2):
        return False
    else:
        for i in range(3,int(n**0.5)+1,2):
            if n%i==0:
                return False
        return True

In [13]:
prompt=hub.pull(owner_repo_commit="hwchase17/openai-tools-agent")

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [17]:
agent=create_openai_tools_agent(llm=llmOpenAI,tools=tools, prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [18]:
question="What is the product of the 998th, 999th and 1000th prime numbers?"

In [19]:
agentExecutor.invoke(input={"input":question})



> Entering new AgentExecutor chain...

Invoking: `GetPrime` with `{'n': 998}`


7901
Invoking: `GetPrime` with `{'n': 999}`


7907
Invoking: `GetPrime` with `{'n': 1000}`


7919The 998th prime number is 7901, the 999th prime number is 7907, and the 1000th prime number is 7919. 

The product of these prime numbers is:
7901 * 7907 * 7919 = 497287193
The product of the 998th, 999th, and 1000th prime numbers is 497287193.

> Finished chain.


{'input': 'What is the product of the 998th, 999th and 1000th prime numbers?',
 'output': 'The 998th prime number is 7901, the 999th prime number is 7907, and the 1000th prime number is 7919. \n\nThe product of these prime numbers is:\n7901 * 7907 * 7919 = 497287193\nThe product of the 998th, 999th, and 1000th prime numbers is 497287193.'}

In [126]:
for step in agentExecutor.iter(inputs={"input":question}):
    print(step)
    print("--------")



> Entering new None chain...

Invoking: `GetPrime` with `{'n': 998}`


7901{'intermediate_step': [(ToolAgentAction(tool='GetPrime', tool_input={'n': 998}, log="\nInvoking: `GetPrime` with `{'n': 998}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_iO2gZYD6vVUERNz1WLpWNdaR', 'function': {'arguments': '{\n  "n": 998\n}', 'name': 'GetPrime'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4-0613'}, id='run-ae5517a6-c50e-49e0-9fdb-eac6ffdee293', tool_calls=[{'name': 'GetPrime', 'args': {'n': 998}, 'id': 'call_iO2gZYD6vVUERNz1WLpWNdaR', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'GetPrime', 'args': '{\n  "n": 998\n}', 'id': 'call_iO2gZYD6vVUERNz1WLpWNdaR', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_iO2gZYD6vVUERNz1WLpWNdaR'), '7901')]}
--------

Invoking: `GetPrime` with `{'n': 999}`


7907{'intermediate_step': [(ToolAgentAction(tool='GetPrime', tool_input={'n

In [123]:
for step in agentExecutor.iter(inputs={"input":question}):
    if step.get('intermediate_step'):
        output=step['intermediate_step']
        action,value=output[0]
        if action.tool=="GetPrime":
            value=int(value)
            print(f"Checking whether {value} is Prime")
            assert isPrime(value)
        # # Ask users if they would want to continue
        _continue=input("Should the agent continue (Y/n)") or "Y"
        if _continue.lower()!="y":
            break
        print("--------")



> Entering new None chain...

Invoking: `GetPrime` with `{'n': 998}`


7901Checking whether 7901 is Prime


Should the agent continue (Y/n) y


--------

Invoking: `GetPrime` with `{'n': 999}`


7907Checking whether 7907 is Prime


Should the agent continue (Y/n) y


--------

Invoking: `GetPrime` with `{'n': 1000}`


7919Checking whether 7919 is Prime


Should the agent continue (Y/n) y


--------

Invoking: `Calculator` with `{'question': '7901 * 7907 * 7919'}`




> Entering new LLMMathChain chain...
7901 * 7907 * 7919```text
7901 * 7907 * 7919
```
...numexpr.evaluate("7901 * 7907 * 7919")...

Answer: 494725326233
> Finished chain.
{'question': '7901 * 7907 * 7919', 'answer': 'Answer: 494725326233'}

Should the agent continue (Y/n) y


--------
The product of the 998th, 999th and 1000th prime numbers is 494725326233.

> Finished chain.


<h3> Doing Similar stuffs with Stream</h3>

In [154]:
for step in agentExecutor.stream(input={"input":question}):
    if step.get('steps'):
        output=step['steps']  # Replace intermediate_step with step
        action,value=output[0]
        action=action[1]  # Extra Step
        if action.tool=="GetPrime":
            value=value[1]  # Extra Step
            value=int(value)
            print(f"Checking whether {value} is Prime")
            assert isPrime(value)
        # # Ask users if they would want to continue
        _continue=input("Should the agent continue (Y/n)") or "Y"
        if _continue.lower()!="y":
            break
        print("--------")



> Entering new None chain...

Invoking: `GetPrime` with `{'n': 998}`


7901Checking whether 7901 is Prime


Should the agent continue (Y/n) y


--------

Invoking: `GetPrime` with `{'n': 999}`


7907Checking whether 7907 is Prime


Should the agent continue (Y/n) y


--------

Invoking: `GetPrime` with `{'n': 1000}`


7919Checking whether 7919 is Prime


Should the agent continue (Y/n) y


--------

Invoking: `Calculator` with `{'question': '7901 * 7907 * 7919'}`




> Entering new LLMMathChain chain...
7901 * 7907 * 7919```text
7901 * 7907 * 7919
```
...numexpr.evaluate("7901 * 7907 * 7919")...

Answer: 494725326233
> Finished chain.
{'question': '7901 * 7907 * 7919', 'answer': 'Answer: 494725326233'}

Should the agent continue (Y/n) y


--------
The product of the 998th, 999th and 1000th prime numbers is 494725326233.

> Finished chain.
